<h1>sentinelTiler.ipynb </h1>

Used to read in and augment sentinel 2 data, providing 128x128x3 image tiles that reflect the GSD of NICSAT.

<b> General Info: </b>
- https://stackoverflow.com/questions/55078484/open-jupyter-notebook-from-a-drive-other-than-c-drive For getting setup on this drive
- https://www.youtube.com/watch?v=3kj8uoOlwjg for tutorial on reading in sentinel data
- https://en.wikipedia.org/wiki/Sentinel-2 for Sentinel colour channels

Sentinel 2 data can be accessed and downloaded from the <a href="https://www.copernicus.eu/en/accessing-data-where-and-how/conventional-data-access-hubs">Copernicus Open Access hub</a>. (SCI Hub)

Once downloaded, extract the files from the zip titled "L2A_T*****_A******", and place them in the designated srcPath.

This code was ran using:

 GDAL: 3030100 
 Rasterio 3.3.1
 
 Multiple issues were encountered with implementing GDAL within the Conda environment (Python 3.9) The user may wish to attempt an install via pip instead.

<h4> Import Libraries </h4>

In [1]:
#import libraries

import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline
import math

from osgeo import gdal, osr
#import gdal
gdal.UseExceptions()

import rasterio
from rasterio import plot
from rasterio.plot import show
from rasterio.enums import Resampling

# Masking libs
#import shapefile as shp
import seaborn as sns
import fiona
import geopandas as gpd
import rasterio.mask
from rasterio.warp import calculate_default_transform, reproject, Resampling

import os
import shutil
import glob
import pandas as pd
from datetime import datetime

#import libtiff
import imageio
import cv2


print('libraries initalised. Version info:\n GDAL:',gdal.VersionInfo(),'\n Rasterio',rasterio.gdal_version())

libraries initalised. Version info:
 GDAL: 3030100 
 Rasterio 3.3.1


In [2]:
# Establish variables
srcPath = 'F:/_code/data/toread/'
tmpPath = 'F:/_code/data/tmp/'

training = True
imWidth = 128
imHeight = 128

print("Accessing Data from:", srcPath, "\nStoring temp files to:" tmpPath)

SyntaxError: invalid syntax (<ipython-input-2-203b0bf1dcb7>, line 8)

In [3]:
# empty temp folder to rid last iteration products
tmpFiles = glob.glob(tmpPath+'*')

for file in tmpFiles:
    os.remove(file)

In [4]:
# define which subfolder to read for either training or testing
if training:
    trainVar = "L2/"
    
else:
    trainVar = "tst/"

L2/


<h4> Level 2A Sentinel Products </h4>

Level 2 products come preporcessed with the Sen2Cor algorithm, so can bypass this part of preprocessing.

Their TOA (Top of Atmosphere) reflectance value have been adjusted to represent BOA (Bottom of Atmosphere) values.

Sen2Cor is available from within QGIS software, or aviualbale from ESA's Science Toolbox.

https://www.qgis.org/en/site/forusers/download.html
https://step.esa.int/main/snap-supported-plugins/sen2cor/

In [5]:
# set up file paths for inputs
# get all Bxx.jp2 files from directories
L2srcPath = glob.glob(srcPath +  trainVar+'/*/')
L2inPath =  glob.glob(srcPath + trainVar+ '/*/IMG_DATA/R10m/')
L2inFiles = [os.path.normpath(pth) for pth in L2inPath]
print('\nFiles Found:\n\n',L2inFiles)


F:/_code/data/toread/

 Files Found:
 ['F:\\_code\\data\\toread\\L2\\L2A_T28UGD_A022071_20210528T115400\\IMG_DATA\\R10m', 'F:\\_code\\data\\toread\\L2\\L2A_T29TQJ_A022614_20210705T110816\\IMG_DATA\\R10m', 'F:\\_code\\data\\toread\\L2\\L2A_T29ULU_A022071_20210528T115400\\IMG_DATA\\R10m', 'F:\\_code\\data\\toread\\L2\\L2A_T29UQR_A031208_20210613T112447\\IMG_DATA\\R10m', 'F:\\_code\\data\\toread\\L2\\L2A_T30TVP_A022614_20210705T110816\\IMG_DATA\\R10m', 'F:\\_code\\data\\toread\\L2\\L2A_T30TVS_A022328_20210615T111721\\IMG_DATA\\R10m', 'F:\\_code\\data\\toread\\L2\\L2A_T32TLL_A031522_20210705T102652\\IMG_DATA\\R10m', 'F:\\_code\\data\\toread\\L2\\L2A_T33SUD_A031536_20210706T095401\\IMG_DATA\\R10m']


In [6]:
#Note, this only works for level 2A products, which have been adjusted for BOA reflectance, rather than TOA...

iter = 0
if iter < len(L2inFiles):
    for folder in L2srcPath:
    
        # import RGB channels from S2 image data. (S2 has 12 bands, we only need 3 for RGB)
        fileR = [os.path.normpath(pth) for pth in glob.glob(L2inPath[iter]+'*_B02_10m.jp2',recursive=False)]
        fileG = [os.path.normpath(pth) for pth in glob.glob(L2inPath[iter]+'*_B03_10m.jp2',recursive=False)]
        fileB = [os.path.normpath(pth) for pth in glob.glob(L2inPath[iter]+'*_B04_10m.jp2',recursive=False)]
        bandR = rasterio.open(fileR[0], driver = 'JP2OpenJPEG')
        bandG = rasterio.open(fileG[0], driver = 'JP2OpenJPEG')
        bandB = rasterio.open(fileB[0], driver = 'JP2OpenJPEG')
        iter = iter + 1
        
        # formulate a tif RGB image using the three seperate band channels
        # as all bands have the same parameters, we can just use a given band to establish the TIF image.
        today = datetime.today()
        outName = today.strftime('IMG_SN_L2_%Y%m%d%H%M%S.tiff')
        with rasterio.open(tmpPath+outName,'w',driver='GTiff',
                          width = bandR.width, height = bandR.height,
                          count = 3,
                          crs=bandR.crs,
                          transform=bandR.transform,
                          dtype=bandR.dtypes[0]
                         ) as src:

            src.write(bandR.read(1),3) # red band (inversed RGB)
            src.write(bandG.read(1),2) # green band (inversed RGB)
            src.write(bandB.read(1),1) # blue band (inversed RGB)
            src.close()
            
        bandR.close()
        bandG.close()
        bandB.close()
 

<h4> Level 1C Sentinel Products </h4>
TOA reflectance

<h2>NICSAT Image Specificiation</h2>

Now that we have the image represented as an RGB tiff file, we need to augment it to reflect NICSAT's imaging system. An overview of the camera specification is given below.
<p>

<table style="width:100%"; table align="left">
    
    <tr> 
        <th>Parameter   <\th>
        <th colspan="1"> Sentinel-2 </th>
        <th colspan="1"> NICSAT </th>
        

  <tr>
    <td>GSD</td>
      <td>10m @ 786km</td>
    <td>42.9m @ 550km &#9733;</td>
    
  </tr>
  <tr>
    <td>Swath Width</td>
      <td>290km @ 786km</td>
    <td>88km @ 550km</td>
    
  </tr>
  <tr>
   <td>Spectral Resolution</td>
      <td>12 Channel</td>
   <td>R,G,B</td>
   
  </tr>
    <tr>
   <td>Focal Length</td>
   <td>600mm </td>
        <td>70mm</td>
   
  </tr>
  <tr>
   <td>F-No.</td>
      <td>F4 (calculated)</td>
   <td>F4</td>
   
  </tr>
   <tr>
   <td>MTF</td>
       <td><a href="https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-2-msi/performance">0.15 to 0.3</a></td>
   <td>~40%</td>
   
  </tr>
    <tr>
   <td>SNR</td>
        <td><a href="https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/resolutions/radiometric">142-168</a></td>
   <td>80-90 &#9733 &#9733</td>
   
  </tr>
</table>
</p>
<p>
<br>
<h4> Notes: </h4>

&#9733; (66m when in de-bayered mode)
<br>
&#9733; &#9733; Typical SNR for a clear scene at AM1.5 standard illumination and 0.3 albedo: 80-90
</p>

<h4>GSD Augmentation </h4>

First, we need to reduce the GSD to reflect the NICSAT System

In [7]:
# open tifs for augmenting in the augment folder
scaleFactor = 0.2331 # scaling factor for 42.9m GSD
#scaleFactor = 1 # scaling factor for GSD

# define output file names
augPath = glob.glob(tmpPath+'*')
augFiles = [os.path.normpath(pth) for pth in augPath]



for file in augFiles:
    with rasterio.open(file) as src:
    
        # resample data
        imgData = src.read(
            out_shape = (src.count, int(src.height * scaleFactor), int(src.width * scaleFactor)),
            resampling = Resampling.bilinear
        )
    
        today = datetime.today()
        outName = today.strftime('GSD_IMG_SN_%Y%m%d%H%M%S.tiff')
        with rasterio.open(tmpPath+outName,'w',driver='GTiff',
                          width = src.width, height = src.height,
                          count = 3,
                          crs=src.crs,
                          transform = src.transform,
                          dtype=src.dtypes[0]
                         ) as outp:

            outp.write(imgData) # red band (inversed RGB)
            outp.close()
            
        src.close()
        os.remove(file) # delete source file from previous step (saves drive space)

<h4>Sea-Land Segmentation</h4>

Uses a buffered land-mask shape file to subtract land from input images, leaving only sea behind. Buffering used to remove beaches/piers.

Current buffer size is 1km using the 1kb shapefile that was created using QGIS software. It was assumed that terrestrial surveillance technologies (coastal radar, AIS, patrols) would manage to survey close inland waters.

In [8]:
# import shapefile for land masking
shpPath = "spprt/shp/ecs_1kb.shp"
shapeFile = gpd.read_file(shpPath)

inPath = glob.glob(tmpPath+'*')
inFiles = [os.path.normpath(pth) for pth in inPath]

for file in inFiles:
    today = datetime.today()
    # coordinate system alignment https://epsg.io/
    ds_crs ='EPSG:3035'     # desired crs type (same as shapefile to perform crop).
    im_out = tmpPath+today.strftime('MSK_IMG_SN_%Y%m%d%H%M%S.tiff')           # output image file (set to overwrite input file to save space)
    
    with rasterio.open(file) as imagery:
        
        tf, x, y = calculate_default_transform(imagery.crs, ds_crs, imagery.width, imagery.height, *imagery.bounds)
        kwargs = imagery.meta.copy()
        kwargs.update({'crs': ds_crs, 'transform': tf, 'width': x, 'height': y})
        
        with rasterio.open(im_out, 'w', **kwargs) as dst:
            for i in range(1, imagery.count + 1):
                reproject(
                    source=rasterio.band(imagery, i),
                    destination=rasterio.band(dst, i),
                    src_transform=imagery.transform,
                    src_crs=imagery.crs,
                    dst_transform=tf,
                    dst_crs=ds_crs,
                    resampling=Resampling.nearest)
            dst.close()
        imagery.close()
    
                
    # clip raster
    with fiona.open(shpPath) as shapeFile:
        shapes = [feature["geometry"] for feature in shapeFile]
        shapeFile.close()
        
    # read img
    with rasterio.open(im_out) as src:
        out_image, out_tf = rasterio.mask.mask(src, shapes, invert=True, pad=True)
        out_meta = src.meta
        src.close()
        
    # save clipped img
    out_meta.update({"driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_tf})
    
    with rasterio.open(im_out,"w",**out_meta) as dest:
        dest.write(out_image)
        dest.close()
     
    os.remove(file) # delete source file from previous step (saves drive space)
        

In [ ]:
# add photometric interpretation (tag262)

inPath = glob.glob(tmpPath+'*')
inFiles = [os.path.normpath(pth) for pth in inPath]

for file in inFiles:
    comstring = ("tiffset -s 262 1 " + file)
    os.system(comstring)
    

In [ ]:
# Convert tiffs to 8bit pngs (saves drive space, but sacrifices 16bit colour channels)
inPath = glob.glob(tmpPath+'*')
outPath = 'F:/_code/data/tiles/pngs/'

inFiles = [os.path.normpath(pth) for pth in inPath]
i = 0
for file in inFiles:
    name = "{0:0=5d}".format(i)
    print('PNG Conversion in progress. File', i, 'of', len(inFiles), '.\n  File:', file, '(IN PROGRESS)')
    img = imageio.imread(file)

    img_norm = np.zeros((img.shape))   
    beta = 4*(img.ptp() / math.sqrt(img.std()))
    
    try:
        img_norm = cv2.normalize(img, img_norm, 0, beta, cv2.NORM_MINMAX, cv2.CV_8UC3)
        fileName = outPath + str(name) + '.png'
        imageio.imwrite(fileName, img_norm)
        print('  File:', file, '(COMPLETE)\n')
        
    except:
        print('  File:', file, '(FAILED)\n')
              
    i = i+1
    img = [] # clear img
    img_norm = []
print('Operation Succeded. ', i, 'files converted.\n')

In [ ]:
# Adapted from GitHub:
# https://github.com/Devyanshu/image-split-with-overlap/blob/master/split_image_with_overlap.py

inPath = 'F:/_code/data/tiles/pngs/'
inPath = glob.glob(inPath+'*')
inFiles = [os.path.normpath(pth) for pth in inPath]
print(inFiles)
outPath = 'F:/_code/data/tiles/128tiles/'

# empty existing tiles
tmpFiles = glob.glob(outPath+'*')
for file in tmpFiles:
    os.remove(file)

def start(size, split_size, overlap=0):
    
    points = []
    
    print(size, split_size, overlap)
    stride = (split_size * (1-overlap))
    stride = int(stride)
    counter = 1
    
    while True:
        pt = stride * counter
        
        if pt + split_size >= size:
            points.append(size - split_size)
            break
        else:
            points.append(pt)
        counter += 1
    return points

split_width = imWidth
split_height = imHeight

count = 0
name = 'split_'
frmt = '.png'

for file in inFiles:
    img = cv2.imread(file)
    img_h, img_w, bnds = img.shape

    X_points = start(img_w, split_width, 0.1)
    Y_points = start(img_h, split_height, 0.1)
    for i in Y_points:
        for j in X_points:
            split = img[i:i+split_height, j:j+split_width]
            fileName = outPath + file[25:-4] + '_{}{}{}'.format(name,count,frmt)
            cv2.imwrite(fileName, split)
            count +=1

In [ ]:
# delete images which are mostly black (performed by investigating file size)
# This may need to be adjusted depending on source image / resolution

inPath = 'F:/_code/data/tiles/128tiles/'
inPath = glob.glob(inPath+'*')
inFiles = [os.path.normpath(pth) for pth in inPath]

for file in inFiles:
    f_size = os.stat(file).st_size
    if f_size <1200:  # 1.9kB
        os.remove(file)

In [ ]:
# empty temp folder after completion (saves drive space)
tmpFiles = glob.glob(tmpPath+'*')

for file in tmpFiles:
    os.remove(file)


<h4> Export Data </h4>

In [ ]:
# import necessary libraries
import glob
import os
import pickle
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
import imageio
import PIL


In [ ]:
# custom function for randomly rotating an image by 90 degrees for augmentation
# declare variables
dataset = []

# declare functions

def create_dataset(source):     # function for producing a dataset by reading image data as an array, and appending it to an array ready for serialisation
    
    for img in source:
        
        img_array = cv2.imread(img)  # read in png into array
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        dataset.append(img_array)
        
    return dataset

def orthogonal_rot(image):     # simple function for rotating an image by a random choice of 0, 90, or -90 degrees
    
    return np.rot90(image, np.random.choice([-1, 0, 1]))



In [ ]:
# perform augmentation
outpth = "F:/_code/data/128Aug/train/"
from tensorflow.keras.preprocessing import image as tfi
def data_augment(fileIn, nameint, counter=100):
    img = cv2.imread(fileIn)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.uint8) 
    img = np.expand_dims(img, axis=0) # convert to a 4D tensor
    
    # construct image gernerator for augmentation
    augr = ImageDataGenerator(
        zoom_range=[1,1],
        preprocessing_function=orthogonal_rot,
        fill_mode="wrap",
        horizontal_flip=False,
        vertical_flip=True,
        channel_shift_range = 10)
        #brightness_range = [0.3, 0.6])
    
    name = "img" + str(nameint).zfill(6)
    name = str(name)
    save_to_dir=outpth
    #imageGen = augr.flow(img, batch_size=1, save_to_dir=outpth, save_prefix=name, save_format="png")
    imageGen = augr.flow(img, batch_size=1, save_format=".png")
    i = 0
    for batch in imageGen:
        img_sv = tfi.array_to_img(batch[0],scale=False)
        img_sv.save(save_to_dir + name + fr"_{i}.png")
        if (i == sampleNo - 1):
            break
        i +=1
    
    total = 0
    for image in imageGen:
        total += 1
        if total == counter:
            break
    

print(PIL.__version__)  # an error can happem with brightness range if PIL is 8.3.0...          

In [ ]:
src_path = glob.glob("F:/_code/GitHub/IRP/data/128/train_new/*")
print(len(src_path))
sampleNo = 5
count = 0
for file in src_path:
    
    data_augment(file, count, sampleNo)
    count = count + 1
    
print('All Images Augmented.')